In [0]:
#We are using Google COLAB(Colaboratory)
#importing modules
#The tale of 2 cities begins here
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

from statsmodels.tsa.ar_model import AR
import pandas as pd
import random 
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from matplotlib import pyplot as mat
from statsmodels.tsa.api import VAR
from statsmodels.stats.stattools import durbin_watson
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#reading csv
data_org=pd.read_csv("/content/drive/My Drive/projectDeng/trains.csv")
datal_org=pd.read_csv("/content/drive/My Drive/projectDeng/label.csv")
data_tes=pd.read_csv("")
datat=data_org.copy()
datal=datal_org.copy()
datat.insert(23,"total_cases",datal.total_cases,True)
datat


FileNotFoundError: ignored

In [0]:
#splitting data based on the city
data_sj=datat[datat['city']=='sj']
data_sj['week_start_date']=pd.to_datetime(data_sj['week_start_date'])
data_sj.set_index(['week_start_date'],inplace=True)
data_iq=datat[datat['city']=='iq']
data_iq['week_start_date']=pd.to_datetime(data_iq['week_start_date'])
data_iq.set_index(['week_start_date'],inplace=True)

In [0]:
data_sj.isnull().values

In [0]:
#Remove null values from the dataset by adding mean of adjecent terms as the missing element
def removenull(dataq):
  for col in dataq:
    if(col!="city" and col!="week_start_date"):
      num_of_rows=dataq['city'].count()
      for i in range(0,num_of_rows-2):
        if(pd.isna(dataq.iloc[i][col])):
          f_found=i
          first=i-1
          countna=0
          for j in range(f_found,num_of_rows):
            countna=countna+1
            if(not pd.isna(dataq.iloc[j][col])):
              l_found=j
              print(dataq[col][f_found],dataq[col][l_found])
              dataq[col][f_found:l_found].fillna((dataq.iloc[first][col]+dataq.iloc[l_found][col])/2,inplace=True)
              break
  return dataq
data_sj=removenull(data_sj)
data_iq=removenull(data_iq)
data_sj[data_sj['ndvi_ne'].isnull().values]

In [0]:
#visualization for tthe city sj(san juan)
dataset_det={'station_max_temp_c' : 'Maximum temperature',
'station_min_temp_c' : 'Minimum temperature',
'station_avg_temp_c': 'Average temperature',
'station_precip_mm' :'Total precipitation',
'station_diur_temp_rng_c' : 'Diurnal temperature range',
'precipitation_amt_mm':'Total precipitation',
'reanalysis_sat_precip_amt_mm':'Total precipitation',
'reanalysis_dew_point_temp_k': 'Mean dew point temperature',
'reanalysis_air_temp_k' :'Mean air temperature',
'reanalysis_relative_humidity_percent' : 'Mean relative humidity',
'reanalysis_specific_humidity_g_per_kg' : 'Mean specific humidity',
'reanalysis_precip_amt_kg_per_m2' : 'Total precipitation',
'reanalysis_max_air_temp_k' : 'Maximum air temperature',
'reanalysis_min_air_temp_k' : 'Minimum air temperature',
'reanalysis_avg_temp_k' :'Average air temperature',
'reanalysis_tdtr_k' : 'Diurnal temperature range',
'ndvi_se' : 'Pixel southeast of city centroid',
'ndvi_sw' : 'Pixel southwest of city centroid',
'ndvi_ne' : 'Pixel northeast of city centroid',
'ndvi_nw' : 'Pixel northwest of city centroid',
'total_cases':'total no of cases'}
set_of_colors=['blue','darkgreen','brown','royalblue','maroon','darkviolet','slateblue','indigo']

d_columns=data_sj.columns
for col in d_columns:
  if col not in ['year','weekofyear','city']:
    dfo=data_sj[col]
    mat.style.use('ggplot')
    mat.plot(dfo.index,dfo,color=random.choice(set_of_colors))
    mat.xlabel('Year') 
    mat.ylabel(col)
    mat.title(col+' vs Time')
    mat.show()
    print("\n",col + " is the "+ dataset_det[col]+"\n\n")



In [0]:
#visualization for the city iq(Iquitos)
set_of_colors=['blue','darkgreen','brown','royalblue','maroon','darkviolet','slateblue','indigo']

d_columns=data_iq.columns
for col in d_columns:
  if col not in ['year','weekofyear','city']:
    dfo=data_iq[col]
    mat.style.use('ggplot')
    mat.plot(dfo.index,dfo,color=random.choice(set_of_colors))
    mat.xlabel('Year') 
    mat.ylabel(col)
    mat.title(col+' vs Time')
    mat.show()

    print("\n",col + " is the "+ dataset_det[col]+"\n\n")



In [0]:
data_sj.total_cases.hist(bins=40)

In [0]:
data_iq.total_cases.hist(bins=40)

In [0]:
#it take 1 week for a dengue infected to show symptoms also it take approximately 2 weeks for a mosquito to become adult
#In total we have to shift 3 weeks so that our result can be more accurate
print(data_sj)
data_sj.total_cases=data_sj.total_cases.shift(-3,axis=0)
data_sj

In [0]:
#remove last 3 rows because we dont have the total cases for it
data_sj.drop(data_sj.index[[-1,-2,-3]],inplace=True)

In [0]:
sj_correlation=data_sj.corr()
mat.figure(figsize=(10,8))

sj_corr_heat = sns.heatmap(sj_correlation,cmap="YlGnBu",linewidths=.1,linecolor='black')

mat.title('San Juan Variable Correlations')


In [0]:
#it take 1 week for a dengue infected to show symptoms also it take approximately 2 weeks for a mosquito to become adult
#In total we have to shift 3 weeks so that our result can be more accurate
print(data_iq)
data_iq.total_cases=data_iq.total_cases.shift(-3,axis=0)
data_iq

In [0]:
#remove last 3 rows because we dont have the total cases for it
data_iq.drop(data_iq.index[[-1,-2,-3]],inplace=True)

In [0]:
#finding correlation of train data sj
iq_correlation=data_iq.corr()
mat.figure(figsize=(10,8))

iq_corr_heat = sns.heatmap(iq_correlation,cmap="YlGnBu",linewidths=.1,linecolor='black')
mat.title('iquitos Variable Correlations')

In [0]:
mat.figure(figsize=(10,8))
(sj_correlation
     .total_cases
     .drop('total_cases')
     .drop('weekofyear')
     .drop('year') 
     .sort_values(ascending=False)
     .plot
     .barh())

In [0]:
mat.figure(figsize=(10,8))
(iq_correlation
     .total_cases
     .drop('total_cases')
     .drop('weekofyear')
     .drop('year') 
     .sort_values(ascending=False)
     .plot
     .barh())

In [0]:
mat.style.use('ggplot')
mat.scatter(data_sj.precipitation_amt_mm,data_sj.total_cases,color=random.choice(set_of_colors))
mat.xlabel('precipitation_amt_mm') 
mat.ylabel('total_cases')
mat.title('precipitation_amt_mm'+' vs total_cases')
mat.show()

In [0]:
mat.figure(figsize=(10,8))
mat.style.use('ggplot')
mat.hist(data_sj['ndvi_ne'],bins=100,alpha=.5)
mat.hist(data_sj['ndvi_se'],bins=100,alpha=.5)
mat.hist(data_sj['ndvi_nw'],bins=100,alpha=.7)
mat.hist(data_sj['ndvi_sw'],bins=100,alpha=.5)


In [0]:

#df3=pd.merge(df1,df2,how='inner')
mat.scatter((data_sj.ndvi_ne+data_sj.ndvi_ne+data_sj.ndvi_ne+data_sj.ndvi_ne)/4,data_sj.total_cases)

In [0]:
mat.style.use('ggplot')
mat.scatter(data_sj.reanalysis_relative_humidity_percent,data_sj.total_cases,color=random.choice(set_of_colors))
mat.xlabel('reanalysis_relative_humidity_percent') 
mat.ylabel('total_cases')
mat.title('reanalysis_relative_humidity_percent'+' vs total_cases')
mat.show()

In [0]:
mat.style.use('ggplot')
mat.scatter(data_sj.reanalysis_relative_humidity_percent,data_sj.reanalysis_specific_humidity_g_per_kg,color=random.choice(set_of_colors))
mat.xlabel('reanalysis_relative_humidity_percent') 
mat.ylabel('total_cases')
mat.title('reanalysis_relative_humidity_percent'+' vs reanalysis_specific_humidity_g_per_kg')
mat.show()

In [0]:
mat.style.use('ggplot')
mat.scatter(data_sj.reanalysis_specific_humidity_g_per_kg,data_sj.total_cases,color=random.choice(set_of_colors))
mat.xlabel('reanalysis_relative_humidity_percent') 
mat.ylabel('total_cases')
mat.title('reanalysis_relative_humidity_percent'+' vs total_cases')
mat.show()

In [0]:
#defing the function
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    

In [0]:
# ADF Test on each column
print(data_sj['year'])
data_sj.drop(['year'],axis=1,inplace=True)
data_sj.drop(['weekofyear'],axis=1,inplace=True)
data_sj.drop(['city'],axis=1,inplace=True)


for name, column in data_sj.iteritems() :
  if(name !="city"):
    adfuller_test(column, name=column.name)
    print('\n')

In [0]:
datao=data_sj[['reanalysis_specific_humidity_g_per_kg','reanalysis_dew_point_temp_k','reanalysis_min_air_temp_k','station_min_temp_c']]
df_differenced=datao.values


In [0]:

datao=data_sj[['reanalysis_specific_humidity_g_per_kg','reanalysis_dew_point_temp_k','reanalysis_min_air_temp_k','station_min_temp_c','total_cases','station_avg_temp_c','precipitation_amt_mm','station_precip_mm','reanalysis_air_temp_k']]
df_differenced=datao
nobs = 4
datao, df_test = datao[0:-nobs], datao[-nobs:]

# Check size
  # (4, 8)
print(datao)
model = VAR(df_differenced)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

In [0]:
model_fitted = model.fit(3)
model_fitted.summary()

In [0]:
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

# Input data for forecasting
forecast_input = df_differenced.values[-lag_order:]
forecast_input

In [0]:
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df_differenced.index[-nobs:], columns=datao.columns+"forecasted" )
df_forecast

In [0]:
fig, axes = mat.subplots(nrows=int(len(datao.columns)/2), ncols=2, dpi=150, figsize=(10,10))
for i, (col,ax) in enumerate(zip(datao.columns, axes.flatten())):
    df_forecast[col+"forecasted"].plot(legend=True, ax=ax).autoscale(axis='x',tight=True)
    df_test[col][-nobs:].plot(legend=True, ax=ax);
    ax.set_title(col + ": Forecast vs Actuals")
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

mat.tight_layout();

In [0]:
# ADF Test on each column
#prediction for city iquitos begins here
data_iq.drop(['year'],axis=1,inplace=True)
data_iq.drop(['weekofyear'],axis=1,inplace=True)
data_iq.drop(['city'],axis=1,inplace=True)


for name, column in data_iq.iteritems() :
  if(name !="city"):
    adfuller_test(column, name=column.name)
    print('\n')

In [0]:
datao=data_iq[['reanalysis_specific_humidity_g_per_kg','reanalysis_dew_point_temp_k','reanalysis_min_air_temp_k','station_min_temp_c','total_cases','station_avg_temp_c','precipitation_amt_mm','station_precip_mm','reanalysis_air_temp_k']]
df_differenced=datao
nobs = 4
datao, df_test = datao[0:-nobs], datao[-nobs:]

# Check size
  # (4, 8)
print(datao)
model = VAR(df_differenced)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

In [0]:
model_fitted = model.fit(2)
model_fitted.summary()

In [0]:
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

# Input data for forecasting
forecast_input = df_differenced.values[-lag_order:]
forecast_input

In [0]:
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df_differenced.index[-nobs:], columns=datao.columns+"forecasted" )
df_forecast

In [0]:
fig, axes = mat.subplots(nrows=int(len(datao.columns)/2), ncols=2, dpi=150, figsize=(10,10))
for i, (col,ax) in enumerate(zip(datao.columns, axes.flatten())):
    df_forecast[col+"forecasted"].plot(legend=True, ax=ax).autoscale(axis='x',tight=True)
    df_test[col][-nobs:].plot(legend=True, ax=ax);
    ax.set_title(col + ": Forecast vs Actuals")
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

mat.tight_layout();

In [0]:
print("forecast finished \nThank God \n credits: https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/")